# Spart-part Deduplicate

## Project Overview

Spare-Part Deduplication ด้วย Hybrid RAG และ Graph-based Grouping

โปรเจค Spare-Part Deduplication มีวัตถุประสงค์เพื่อแก้ไขปัญหา material / spare-part ที่ซ้ำกันแต่มีหลายรหัส (PK) ซึ่งเกิดจากการบันทึกข้อมูลจากหลายแหล่ง หลายโรงงาน หรือหลายช่วงเวลา ทำให้ material ที่เป็นชิ้นส่วนเดียวกัน ถูกสร้างเป็นหลาย PK ส่งผลต่อความซับซ้อนของระบบจัดซื้อ คลังสินค้า และการวิเคราะห์ข้อมูลในภาพรวม

แนวคิดและแนวทางแก้ไข (Solution Approach)

โซลูชันของโปรเจคนี้ใช้แนวคิด Retrieval-Augmented Generation (RAG) ในรูปแบบ Hybrid Search เพื่อดึง candidate ที่มีความเป็นไปได้สูงว่าจะเป็น material เดียวกัน จากนั้นใช้ Large Language Model (LLM) เป็นตัวตัดสินเชิงความหมาย (semantic reasoning) ว่า candidate ใดควรถูกจัดเป็น duplicate

diagram flow: (RAG search → LLM → Graph)

ระบบแบ่งการทำงานออกเป็น 3 ขั้นตอนหลัก:
1. Hybrid Search (Candidate Retrieval)

ในขั้นตอนแรก ระบบจะดึง candidate material จากฐานข้อมูล โดยใช้การค้นหาแบบผสม (Hybrid Search) ประกอบด้วย:

Semantic Search (Vector Search)
ใช้ embedding เพื่อจับความหมายของคำอธิบาย material แม้ข้อความจะไม่ตรงกันทุกคำ

Keyword Search (BM25 / Full-text search)
ใช้สำหรับจับ keyword สำคัญ เช่น ประเภทของชิ้นส่วน รหัสรุ่น หรือคำทางเทคนิค

Trigram Fuzzy Matching
ใช้ตรวจจับความคล้ายของข้อความในเชิงตัวอักษร เหมาะกับกรณีสะกดต่างกันเล็กน้อยหรือมีสัญลักษณ์พิเศษ

ผลลัพธ์จากแต่ละวิธีจะถูก normalize และถ่วงน้ำหนัก (weighted scoring) เพื่อคัดเลือก candidate ที่มีแนวโน้มสูงสุดสำหรับขั้นตอนถัดไป

2. LLM-based Grouping (Duplicate Decision)

เมื่อได้ candidate จาก Hybrid Search แล้ว ระบบจะส่งข้อมูลเข้าสู่ LLM เพื่อทำการวิเคราะห์เชิงความหมายและบริบท โดยให้ LLM ตัดสินใจว่า:

material ใดเป็น duplicate ของกันและกัน

material ใดควรถูกแยกเป็นคนละกลุ่ม

LLM จะช่วยจัดกลุ่ม material ที่มีความหมายเหมือนกัน แม้รูปแบบคำอธิบายหรือโครงสร้างข้อมูลจะแตกต่างกัน

3. Graph-based Connectivity Grouping

ผลลัพธ์จาก LLM จะถูกนำไปสร้างเป็น Graph โดย:

material แต่ละ PK คือ node

ความสัมพันธ์ว่า “เป็น duplicate กัน” คือ edge

ระบบจะใช้แนวคิด connected components เพื่อรวมกลุ่ม duplicate ให้ครบถ้วน เช่น:

ถ้า

A = B

B = C

เมื่อนำเข้าสู่ graph จะได้กลุ่มเดียวกันคือ
A = B = C

วิธีนี้ช่วยให้ได้ duplicate group ที่สมบูรณ์ แม้ความสัมพันธ์จะไม่ได้ถูกค้นพบทั้งหมดในครั้งเดียว



ผลลัพธ์ที่คาดหวัง

ลดจำนวน PK ที่ซ้ำซ้อนในระบบ

เพิ่มความถูกต้องของ master data

ลดต้นทุนจากการจัดซื้อซ้ำหรือการจัดเก็บชิ้นส่วนซ้ำ

วางรากฐานสำหรับ data governance และ master data management ในระยะยาว

------
batch ละ 500
design run เป็นรอบ ๆ 500 record → search → LLM 50 parallel → batch ต่อไป

เก็บ stage ลง DB ทุกครั้งแต่ละ step

## Import Lib

In [1]:
import pandas as pd
import re

In [2]:
from google import genai
from google.genai import types
import random
from tqdm import tqdm

## Prepare Data

In [3]:
po = pd.read_csv("Vw_POText.csv", encoding="utf-8")

In [4]:
# รวมข้อความตาม Material
po_combine_text = (
    po.sort_values(["Material", "Line"])                # เรียงตาม Material และ Line
      .groupby("Material")                              # จัดกลุ่มตาม Material
      .agg({"Text": lambda x: " ".join(x.astype(str))}) # รวม Text โดยคั่นด้วย space
      .reset_index()                                     # ให้ Material กลับมาเป็น column
)

po_combine_text

,Material,Text
0,210STKR0000018,STK.PE ใสหลังกลาสซีน กาวธรรมดา 180mmx1000m #BW...
1,225107A400004228,CK SP CK SP33131ORANGE nan ORANGE 4228 nan nan
2,2503,"PLC, MFR PART NAME : PLC CPU, MFR PART NO. : 6..."
3,25032A000045001,"FUSE, SHORT NAME : HCR FUSE 24KV/100A, VOLTAG..."
4,25032A000045002,SENSOR S30B-2011DA#1026822 สถานที่ใช้งาน TRANS...
...,...,...
177924,W410WT140100001,"KRAFT, TYPE : LINER, MFR MODEL NO. : WT140, ..."
177925,W410WT160000001,"KRAFT, TYPE : LINER, MFR MODEL NO. : WT160, ..."
177926,W410YL140130001,KRAFT LINER SDY TL140
177927,W431FORMLW00001,แบบฟอร์มกระดาษต่อเนื่อง ใบกำกับภาษี/ใบส่งของ ก...


In [5]:
mm = pd.read_csv("Vw_MaterialMaster.csv", encoding="utf-8")

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_87130/1059879351.py:1: DtypeWarning: Columns (4,11,14,15,16,17,18,19,20,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  mm = pd.read_csv("Vw_MaterialMaster.csv", encoding="utf-8")


In [6]:
material_type = ["06"]

In [7]:
select_mm_matType = mm[
    mm["Material_Type"]
    .fillna(-1)
    .astype(int)
    .map("{:02d}".format)
    .isin(material_type)
]

In [8]:
mm["pk_plant_matnum"] = (
    mm["Plant"].fillna("").astype(str).str.strip() + "_" +
    mm["Material_Number"].fillna("").astype(str).str.strip()
)

In [9]:
select_mm_matType = mm[mm["Material_Type"] == 6.0]

In [10]:
select_mm_matType_no_dup = select_mm_matType.drop_duplicates(subset=["pk_plant_matnum"], keep="first")

In [11]:
select_mm_matType_join = select_mm_matType_no_dup.merge(
    po_combine_text,
    how="left",
    left_on="Material_Number",
    right_on="Material"
)

In [12]:
select_mm_matType_mapped = select_mm_matType_join[select_mm_matType_join["Material"].notna()]
select_select_mm_matType = select_mm_matType_mapped[["pk_plant_matnum", "Material_Type", "Plant", "Material_Number", "PlantID", "Plant_Description", "UOM", "Material", "Material_Description", "Text"]]
select_select_mm_matType = select_mm_matType_mapped.rename(columns={
    "Material_Description": "material_description"
})

## 0. Combine field

### 0.1 for raw_combine_for_embedding (without uom)

In [13]:
select_select_mm_matType["raw_combine_for_embedding"] = select_select_mm_matType["material_description"] + " " + select_select_mm_matType["Text"]

### 0.2 for raw_combine_for_keyword (add uom)

In [14]:
select_select_mm_matType["raw_combine_for_keyword"] = select_select_mm_matType["material_description"] + " " + select_select_mm_matType["Text"] + " " + select_select_mm_matType["UOM"]

## 1. Data Cleaning & Normalization
สรุป: ทำให้ข้อมูลสะอาด สม่ำเสมอ พร้อมให้ embedding และ BM25 ใช้ได้ดี
- Uppercase ทั้งหมด
- Normalize whitespace
- เก็บ character สำคัญของ part code (- / _ . ( ))
- Remove noise tokens (EA, PCS, ITEM, DESC)
- รวม material_description + PO_text → context

In [15]:
def normalize_whitespace(text: str) -> str:
    # แทน \n \t ด้วย space และลด space ซ้อน
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def remove_duplicate_commas_periods(text: str) -> str:
    # ลด ,, → , และ .. → .
    text = re.sub(r",{2,}", ",", text)
    text = re.sub(r"\.{2,}", ".", text)
    return text

def lowercase_text(text: str) -> str:
    return text.lower()


# --------- Main Cleaning Function ---------
def cleaning_features(text: str) -> str:
    if pd.isna(text):
        return ""

    # 1. normalize whitespace
    text = normalize_whitespace(text)

    # 2. remove duplicate punctuation
    text = remove_duplicate_commas_periods(text)

    # 3. keep punctuation that conveys meaning → (ไม่มีการลบใด ๆ)

    # 4. lowercase for embedding stability
    # text = lowercase_text(text)

    # 5. Normalize dash/underscore variants
    text = normalize_dashes(text)

    return text.strip()

def normalize_dashes(text):
    return text.replace("–", "-").replace("—", "-")

def extract_partcodes(text):
    """Extract part codes/model numbers from text"""
    # Pattern 1: Alphanumeric with separators (e.g., 3310-ATN9, NU-318)
    part_pattern = r"(?=[A-Z0-9]*[A-Z])[A-Z0-9]+(?:[-_/\.][A-Z0-9]+)+"
    parts = re.findall(part_pattern, text.upper())
    
    # Pattern 2: Model numbers (e.g., 3310, 7306)
    model_pattern = r'\b[A-Z0-9]{3,}\b'
    models = re.findall(model_pattern, text.upper())
    
    # Combine and deduplicate
    all_codes = parts + models
    return list(dict.fromkeys(all_codes))

def normalize_comma_spacing(text: str) -> str:
    # บังคับให้มี space หน้าและหลัง "," เสมอ
    return re.sub(r"\s*,\s*", " , ", text).strip()

# --------- Build Embedding Text ---------
def build_raw_combine_for_embedding(row):
    sd = cleaning_features(row["material_description"]) if pd.notna(row["material_description"]) else ""
    ld = cleaning_features(row["Text"]) if pd.notna(row["Text"]) else ""

    combined = f"{sd} {ld}"

    # # extract part-code to reinforce semantic
    # parts = extract_partcodes(combined)
    # if parts:
    #     combined += " part_codes: " + ", ".join(parts)

    # normalize comma spacing
    combined = normalize_comma_spacing(combined)
    
    # lowercase everything for embedding stability
    combined = lowercase_text(combined)

    return combined.strip()

### 1.1 cleansing for sementic search (embed)
- 1. normalize whitespace
- 2. remove duplicate punctuation
- 3. Normalize dash/underscore variants
- 4. extract part-code to reinforce semantic
- 5. lowercase everything for embedding stability

In [16]:
select_select_mm_matType["clean_combine_for_embedding"] = (
    select_select_mm_matType.apply(build_raw_combine_for_embedding, axis=1)
)

### 1.2 cleansing for keyword search

In [17]:
# Function before extracting identifier_keyword
def clean_quotes(text: str) -> str:
    """
    - remove "word" ที่ไม่มีเลขด้านใน
    - remove stray quotes เช่น "zn → zn
    - แปลง inch: 3/4" → 3/4 inch
    - (สุดท้ายค่อย normalize spacing)
    """

    # ---------------------------------------------------------
    # A) remove quoted words without digits → remove entirely
    # "SKF" → space
    # "MAHLE" → space
    # "abc-def" → space
    # ---------------------------------------------------------
    def remove_non_numeric_quotes(m):
        inside = m.group(1)
        if not re.search(r'\d', inside):
            return " "     # ไม่มีเลข → remove
        return m.group(0)  # มีเลข → keep

    text = re.sub(r'"([^"]*)"', remove_non_numeric_quotes, text)

    # ---------------------------------------------------------
    # B) remove stray quotes (quote เดี่ยว ๆ ที่เหลือ)
    # เช่น "zn → zn,  zn" → zn
    # ---------------------------------------------------------
    text = re.sub(r'(^| )"(\w+)', r'\1\2', text)
    text = re.sub(r'(\w+)"( |$)', r'\1\2', text)

    # ---------------------------------------------------------
    # C) convert inch (only numeric + double-quote)
    # ทำหลังจาก clean quotes แล้ว
    # ---------------------------------------------------------
    text = re.sub(r'(\d+/\d+|\d+(?:\.\d+)?)"', r'\1 inch', text)
    text = re.sub(r'(\d+/\d+|\d+(?:\.\d+)?)\s*"(\s|$)', r'\1 inch ', text)

    # ---------------------------------------------------------
    # D) normalize spaces
    # ---------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

special_chars = ["?"]

def clean_special_chars(text: str, special_chars: list) -> str:
    """
    Remove / replace special characters defined in special_chars list.
    Replace with space to preserve token boundaries.
    """
    if not text:
        return text

    # escape chars for regex safety (เช่น ?, *, ., etc.)
    escaped = [re.escape(c) for c in special_chars]
    pattern = f"[{''.join(escaped)}]"

    # replace with space
    text = re.sub(pattern, " ", text)

    # collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


def normalize_spacing(text):
    """
    - ใส่ space ก่อน–หลัง comma
    - compress multiple spaces
    """

    # Add space around commas
    text = re.sub(r'\s*,\s*', ' , ', text)

    # collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def normalize_uom(text: str) -> str:

    # Convert number + optional space + " → number inch
    # Examples:
    #   1/2"  → 1/2 inch
    #   1/2 " → 1/2 inch
    #   10"   → 10 inch
    #   10 "  → 10 inch
    text = re.sub(r'\b(\d+(?:/\d+)?)(\s*)"(?![a-zA-Z])', r'\1 inch', text)

    # --- 2. millimeter → mm ---
    text = re.sub(r'\bmil{0,1}imet(er|re|ers|res)\b', 'mm', text)

    # --- 3. kg variations → kg ---
    text = re.sub(r'\bkg\.?\b', 'kg', text)
    text = re.sub(r'\bkgs?\.?\b', 'kg', text)
    text = re.sub(r'\bkilogram(s)?\b', 'kg', text)

    # --- 4. bars → bar ---
    text = re.sub(r'\bbars\b', 'bar', text)

    # --- 5. centimeter → cm ---
    text = re.sub(r'\bcentimet(er|re|ers|res)\b', 'cm', text)
    text = re.sub(r'\bcentimeter(s)?\b', 'cm', text)

    # --- 6. meter → m ---  
    # (ไม่แตะกรณี M560475D1 เพราะต้องเป็น word boundary)
    text = re.sub(r'\bmet(er|re|ers|res)\b', 'm', text)
    text = re.sub(r'\bmeters\b', 'm', text)

    # --- 7. liter → L ---
    text = re.sub(r'\blitre(s)?\b', 'L', text)
    text = re.sub(r'\bliter(s)?\b', 'L', text)

    # --- 8. ton / tonne ---
    text = re.sub(r'\btonne(s)?\b', 'ton', text)
    text = re.sub(r'\btons\b', 'ton', text)

    return text

# ============================================================
# 2) Preprocess text (normalize punctuation / spacing / dash)
# ============================================================
def normalize_text(t: str) -> str:
    t = t.lower()

    # normalize dash variations
    t = t.replace("–", "-").replace("—", "-").replace("_", "-")

    # remove duplicate punctuations (,,, ---> ,)
    # reduce duplicate punctuation (safe)
    t = re.sub(r'([!?,.;]{2,})', lambda m: m.group()[0], t)

    # # remove unwanted punctuations: * and :
    t = t.replace("*", " ")
    t = t.replace(":", " ")

    # # normalize spaces
    t = re.sub(r"\s+", " ", t).strip()

    return t

def normalize_temperature(text):
    # -------- Celsius --------
    # ครอบคลุม:
    # DEGREE CELSIUS, DEG CELSISU, DEG C., 
    #  C, DEG.CELSIUS,
    # DEGREECELSIUS, DEGREE OF CELSIUS, DEGREE OF C, DEG C.
    celsius_patterns = [
        r'\b(\d+)\s*(deg\.?|degree\.?|degrees\.?)\s*(of\s*)?(celsius|celcius|celsisu|cels|cel|c)\b',
        r'\b(\d+)\s*(°)\s*c\b',
        r'\b(\d+)\s*c\b'
    ]

    for p in celsius_patterns:
        text = re.sub(p, r'\1°C', text, flags=re.I)

    # -------- Fahrenheit --------
    fahrenheit_patterns = [
        r'\b(\d+)\s*(deg\.?|degree\.?|degrees\.?)\s*(of\s*)?(fahrenheit|farenheit|farenhiet|f)\b',
        r'\b(\d+)\s*(°)\s*f\b',
        r'\b(\d+)\s*f\b'
    ]

    for p in fahrenheit_patterns:
        text = re.sub(p, r'\1°F', text, flags=re.I)

    return text

def normalize_technical_terms(text: str) -> str:
    # ------------------------------------------------------------
    # STEP 1) handle dia. followed by number → insert space only
    # ------------------------------------------------------------
    text = re.sub(r'\bdia\.(?=\d)', r'dia. ', text, flags=re.I)

    # ------------------------------------------------------------
    # STEP 2) normalize exact tokens only → dia.
    #   - diameter
    #   - diam.
    #   - dia
    #   - dia.  (เฉพาะกรณีเป็น token เดี่ยว ๆ)
    #   - ø φ Ø Φ
    # ------------------------------------------------------------
    # normalize diameter
    text = re.sub(r'\bdiameter\b', 'dia.', text, flags=re.I)

    # normalize diam.
    text = re.sub(r'\bdiam\.\b', 'dia.', text, flags=re.I)

    # normalize symbol forms
    text = re.sub(r'[ØøφΦ⌀]', 'dia.', text)

    # normalize dia. (แต่ต้องเป็น token เดี่ยว ไม่ติดตัวเลข)
    text = re.sub(r'\bdia\.(?!\d)', 'dia.', text, flags=re.I)

    # ------------------------------------------------------------
    # STEP 3) collapse spaces
    # ------------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def clean_dash(text: str) -> str:
    """
    Replace dash-like characters with space.
    """

    # ------------------------------------------------------------
    # STEP 1) normalize dash variants to space
    #   - hyphen-minus (-)
    #   - en dash (–)
    #   - em dash (—)
    # ------------------------------------------------------------
    text = re.sub(r'[-–—]', ' ', text)

    # ------------------------------------------------------------
    # STEP 2) collapse spaces
    # ------------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def clean_slash(text: str) -> str:
    """
    Replace slash characters with space.
    """

    # ------------------------------------------------------------
    # STEP 1) normalize slash variants to space
    #   - forward slash (/)
    #   - backslash (\)
    # ------------------------------------------------------------
    text = re.sub(r'[\\/]', ' ', text)

    # ------------------------------------------------------------
    # STEP 2) collapse spaces
    # ------------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [18]:
# Temp for extract_identifier_candidates
# ============================================================
# 1) Utility regex
# ============================================================

# แนวๆ CODE/PART NUMBER:
code_patterns = [
    r"[A-Z]{1,4}\d{1,6}[A-Z0-9\-]*",    # ABC1234 / E70340M-K059521
    r"\d{2,4}[A-Z]{1,4}\d{2,6}",        # 512ABC1234
    r"[A-Z]{1,4}\d{2,4}",               # TLFH-4CA (ก่อน normalize)
    r"\d{2,4}[A-Z]{1,4}",               
    # r"\bAH[- ]?\d{3,5}\b",              # AH24034
]

code_regex = re.compile("|".join(code_patterns), re.IGNORECASE)
# ============================================================
# 4) Extract CODE patterns for exact-match boosting
# ============================================================
def extract_codes(t: str):
    return list(set(code_regex.findall(t)))

In [19]:
# Extract identifier keyword
# 1. extract_identifier_candidates

IDENTIFIER_PATTERNS = [

    # 1) Classic material / part code
    r"\b[A-Z]{1,4}[- ]?\d{2,6}[A-Z0-9]*\b",

    # 2) Complex code with separators
    r"\b[A-Z0-9]{2,}[-/][A-Z0-9-/]{2,}\b",

    # 3) Drawing / model no.
    r"\b[A-Z]{1,3}\d{4,}[A-Z0-9/.-]*\b",

    # 4) Metric thread / fastener
    r"\bM\d{2,3}\s*[xX]\s*L?\d{2,4}\b",

    # 5) Dimension compound
    r"\b\d{2,4}\s*[xX]\s*\d{2,5}\s*(mm|cm|m)?\b",

    # 6) Pipe / standard
    r"\bDN\d{2,4}\b",
]

IDENTIFIER_REGEX = re.compile("|".join(IDENTIFIER_PATTERNS), re.IGNORECASE)


def extract_identifier_candidates(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    return list(set(m.group(0) for m in IDENTIFIER_REGEX.finditer(text)))


# 2. normalize_identifier
def normalize_identifier(code: str) -> str:
    c = code.strip().upper()

    # 1) normalize spaces around separators
    c = re.sub(r"\s*-\s*", "-", c)
    c = re.sub(r"\s*/\s*", "/", c)
    c = re.sub(r"\s*[xX]\s*", "x", c)

    # 2) AH 24034 → AH-24034
    c = re.sub(r"\b([A-Z]{1,4})\s+(\d{2,6})\b", r"\1-\2", c)

    # 3) normalize unit casing
    c = re.sub(r"\bMM\b", "MM", c)

    return c

# 3. filter_identifier
def is_valid_identifier(code: str) -> bool:

    # too short
    if len(code) < 4:
        return False

    # pure number
    if code.isdigit():
        return False

    # single dimension like 10MM
    if re.fullmatch(r"\d{1,4}(MM|CM|M)?", code):
        return False

    # must contain at least one digit + one letter
    if not (re.search(r"[A-Z]", code) and re.search(r"\d", code)):
        return False

    return True

def extract_identifier_keywords(text: str) -> list[str]:
    raw_candidates = extract_identifier_candidates(text)

    normalized = [normalize_identifier(c) for c in raw_candidates]

    filtered = [c for c in normalized if is_valid_identifier(c)]

    return sorted(set(filtered))



In [20]:
def remove_no(text):
    """
    remove only stopword: "no."
    """
    tokens = text.split()
    tokens = [t for t in tokens if t.lower() not in ["no."]]
    return " ".join(tokens)

# ============================================================
# 5) Remove stopwords for BM25 keyword clarity
# ============================================================
# def remove_stopwords(t: str) -> str:
#     tokens = t.split()
#     tokens = [tok for tok in tokens if tok not in stopwords]
#     return " ".join(tokens)
stopwords_list = ["short",
    "name",
    "long",
    "description",
    "remark",
    "application",
    "equipment",
    "machine",
    "mfr",
    "maker",
    "model",
    "type",
    "serial",
    "supplier",
    "shop",
    "drawing",
    "dwg.",
    "dwg",
    "no.",
    "number",
    "offer",
    "item",
    "position",
    "use for",
    "used for"
    "used",
    "?",
    "from",
    "part",
    "partno",
    "code",
    "unit",
    "qty.",
    "qty",
    "size",
    "pressure",
    "range"
]

stopwords = set(stopwords_list)

def remove_stopwords(t: str) -> str:
    tokens = t.split()
    cleaned = []
    for tok in tokens:
        tok_clean = tok.strip(",:;")  # ลบ punctuation รอบ ๆ
        if tok_clean and tok_clean not in stopwords:  # <--- ป้องกัน "" 
            cleaned.append(tok_clean)
    return " ".join(cleaned)

# ============================================================
# 6) Create keyword_list field
# ============================================================
def build_keyword_list(clean_text: str, codes: list):
    tokens = clean_text.split()
    return list(sorted(set(tokens + codes)))

In [21]:
# ============================================================
# Main logic for keyword pipeline
# ============================================================
def clean_keyword_pipeline(text: str):
    if not isinstance(text, str):
        return "", [], [], []

    t = text

    # 1) special " cleaner (change " to inch / garbage / remove)
    t = clean_quotes(t)

    # 2) clean special chars
    t = clean_special_chars(t, special_chars)

    # 3) Add space around comma
    t = normalize_spacing(t)

    # # 4) normalize
    t = normalize_text(t)

    # # 5) normalize UOM
    t = normalize_uom(t)

    # # 6) normalize degree celsius and Fahrenheit
    t = normalize_temperature(t)

    # # 7) normalize technical term
    t = normalize_technical_terms(t)

    # clean "-" and "/" for test vector search
    # t = clean_dash(t) # Recall drop
    # t = clean_slash(t) # Recall drop

    # # 8) extract codes
    old_codes = extract_codes(t)
    codes = extract_identifier_keywords(t)

    # # 9) remove no.
    t = remove_no(t)

    # 10) remove stopwords
    t_no_stop = remove_stopwords(t)

    # 5) build keyword list
    keyword_list = build_keyword_list(t_no_stop, codes)
    # codes = ""
    # keyword_list = ""

    return t, codes, old_codes, keyword_list



select_select_mm_matType["clean_combine_for_keyword"], select_select_mm_matType["keyword_codes"], select_select_mm_matType["keyword_old_codes"], select_select_mm_matType["keyword_list"] = zip(
    *select_select_mm_matType["raw_combine_for_keyword"].map(clean_keyword_pipeline)
)

select_select_mm_matType.head()

,Material_Type,Material_Number,material_description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,pk_plant_matnum,Material,Text,raw_combine_for_embedding,raw_combine_for_keyword,clean_combine_for_embedding,clean_combine_for_keyword,keyword_codes,keyword_old_codes,keyword_list
0,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,7560_7506809480106,7506809480106,"SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","SPROCKET,CHAIN,49 T ,1/2 ""(ZN) SPROCKET,CHAIN,...","SPROCKET,CHAIN,49 T ,1/2 ""(ZN) SPROCKET,CHAIN,...","sprocket , chain , 49 t , 1/2 ""(zn) sprocket ,...","sprocket , chain , 49 t , 1/2 inch(zn) sprocke...",[B-001],[],"[0.500, 001, 1, 1/2, 106, 309, 49, 8, B-001, a..."
1,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,7560_7806055050934,7806055050934,"FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE...","FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA FLOWMETER, ...","FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA FLOWMETER, ...","flowmeter , 300l/min , tlfh-4ca , kytola flowm...","flowmeter , 300l/min , tlfh-4ca , kytola flowm...","[300L/MIN, TLFH-4CA]",[300l],"[300L/MIN, 300l/min, TLFH-4CA, flow, flowmeter..."
2,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,7560_54060106035573,54060106035573,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ...",ACID PHOSPHORIC COMERCIAL GRADE ACID PHOSPHORI...,ACID PHOSPHORIC COMERCIAL GRADE ACID PHOSPHORI...,acid phosphoric comercial grade acid phosphori...,acid phosphoric comercial grade acid phosphori...,"[E70340M, E70540M, K059001, K059088, K059249, ...","[e70540m-l014005, e70540m-l014006, e70540m-l01...","[(กรดบัดกรี), 35, 512, E70340M, E70540M, K0590..."
3,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,7560_54060140106258,54060140106258,"SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034 SHORT N...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034 SHORT N...","bearing , acc , withdrawal sleeve , ah 24034 s...","bearing , acc , withdrawal sleeve , ah 24034 s...","[AH-24034, M-180]",[],"[180, 24034, 3, AH-24034, M-180, acc, adapter,..."
4,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,7560_54060140113243,54060140113243,"BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028 BEARING...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028 BEARING...","bearing , acc , withdrawal sleeve , ah 24028 b...","bearing , acc , withdrawal sleeve , ah 24028 b...",[AH-24028],[],"[2, 24028, AH-24028, acc, adapter, ah, bearing..."


## Prepare test Data set

In [22]:
material_type = ["06"]

In [23]:
select_mm_matType_no_dup = select_mm_matType.drop_duplicates(subset=["pk_plant_matnum"], keep="first")
select_mm_matType_join = select_mm_matType_no_dup.merge(
    po_combine_text,
    how="left",
    left_on="Material_Number",
    right_on="Material"
)

In [24]:
edges = pd.read_csv("edges.csv")

# 1) เอา 50 แถวแรก
head50 = edges.head(50)

# 2) เอา random 50 แถว (ไม่ซ้ำกับ head50)
random50 = edges.iloc[50:].sample(n=50, random_state=42)

# 3) รวมกัน
sample100 = pd.concat([head50, random50], ignore_index=True)

sample100.head()


,MODEL_BRAND,Key_mat_1,Plant_1,Mat_1,Key_mat_2,Plant_2,Mat_2
0,3310 ATN9_SKF,7560_75060602310346,7560,75060602310346,7511_75060610300414,7511,75060610300414
1,3311 ANR_SKF,7560_75060602411346,7560,75060602411346,7511_75060610310502,7511,75060610310502
2,7306 BECBM_SKF,7812_78060606306581,7812,78060606306581,7511_75060610805312,7511,75060610805312
3,NU 308 ECP/C3_SKF,7812_78060612308552,7812,78060612308552,7511_75060612500369,7511,75060612500369
4,C 2234 K/C4_SKF,7560_75060612234350,7560,75060612234350,7511_75060612556307,7511,75060612556307


In [25]:
keys1 = set(sample100["Key_mat_1"].unique())
keys2 = set(sample100["Key_mat_2"].unique())

answer_keys = keys1.union(keys2)
print(f"จำนวน unique keys ที่ต้องการ: {len(answer_keys)}")

จำนวน unique keys ที่ต้องการ: 177


In [26]:
mm_keys = set(select_select_mm_matType["pk_plant_matnum"].dropna().unique())
missing_keys = answer_keys - mm_keys
print(missing_keys)
print(f"Missing keys: {len(missing_keys)}")

{'PO21_PO06131335083', 'PO21_PO06138007044', 'PO21_PO06131335343', 'PO11_PO31130352303', 'PO21_PO06131335550', 'PO21_PO06131335105', 'PO21_PO06138006899', '7511_75313647080'}
Missing keys: 8


In [27]:
before = len(sample100)

sample100 = sample100[
    sample100["Key_mat_1"].isin(mm_keys) &
    sample100["Key_mat_2"].isin(mm_keys)
].copy()

after = len(sample100)

print(f"sample100 before: {before}")
print(f"sample100 after : {after}")
print(f"dropped records : {before - after}")

sample100 before: 100
sample100 after : 93
dropped records : 7


In [28]:
# สร้าง mask ว่า record ไหนผ่านเงื่อนไข (ต้องมีทั้งคู่) (key ต้องไม่ missing)
mask = (
    sample100["Key_mat_1"].isin(mm_keys) &
    sample100["Key_mat_2"].isin(mm_keys)
)

# นับจำนวน record ที่ผ่าน
count_valid_records = mask.sum()

print("จำนวน record ที่มี Key_mat_1 และ Key_mat_2 ครบทั้งคู่:", count_valid_records)


จำนวน record ที่มี Key_mat_1 และ Key_mat_2 ครบทั้งคู่: 93


In [29]:
# ---------------------------
# 1) เตรียม key 100 ค่า
# ---------------------------
keys_200 = pd.concat([
    sample100["Key_mat_1"],
    sample100["Key_mat_2"]
]).dropna().unique()

print(f"จำนวน unique keys: {len(keys_200)}")

# ---------------------------
# 2) ดึง record จาก table หลัก
# ---------------------------
df_answers = select_select_mm_matType[
    select_select_mm_matType["pk_plant_matnum"].isin(keys_200)
]

print(f"จำนวน record จาก key_mat (answers): {len(df_answers)}")

# ---------------------------
# 3) ตัด record เหล่านี้ออกจาก pool
# ---------------------------
remaining_pool = select_select_mm_matType[
    ~select_select_mm_matType["pk_plant_matnum"].isin(df_answers["pk_plant_matnum"])
]

# ---------------------------
# 4) สุ่มเพิ่ม 200 record (lock seed)
# ---------------------------
df_400 = remaining_pool.sample(
    n=400,
    random_state=42   # 🔒 lock seed
)

print(f"จำนวน random records: {len(df_400)}")

# ---------------------------
# 5) รวมเป็น sample size 300
# ---------------------------

sample_500 = pd.concat([df_answers, df_400], ignore_index=True)

print(f"จำนวน sample ทั้งหมด: {len(sample_500)}")

จำนวน unique keys: 163
จำนวน record จาก key_mat (answers): 163
จำนวน random records: 400
จำนวน sample ทั้งหมด: 563


## DB (RAG)

In [30]:
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev,public -c client_encoding=UTF8"  # Set default schema and UTF-8 encoding
}

In [31]:
import psycopg2
import psycopg2.extras
from pgvector.psycopg2 import register_vector
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# ============================================================
# Configuration
# ============================================================
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev,public -c client_encoding=UTF8"  # Set default schema and UTF-8 encoding
}

TABLE_NAME = "spare_part_test"
SCHEMA_NAME = "apm_dev"
EMBEDDING_DIM = 1024  # bge-m3 dimension

# ============================================================
# Database Connection
# ============================================================
def get_connection():
    """Get database connection with pgvector support"""
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_client_encoding('UTF8')  # Ensure UTF-8 encoding for Thai text
    register_vector(conn)
    return conn

# ============================================================
# 1. CREATE TABLE
# ============================================================
def create_spare_part_table(drop_if_exists: bool = False):
    """
    Create spare_part_test table with all required fields
    
    Args:
        drop_if_exists: If True, drop existing table before creating
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    try:
        # Drop table if requested
        if drop_if_exists:
            print(f"⚠️  Dropping existing table {SCHEMA_NAME}.{TABLE_NAME}...")
            cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_NAME}.{TABLE_NAME} CASCADE")
        
        # Create table
        create_sql = f"""
        CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
            -- Primary Key
            pk_plant_matnum TEXT PRIMARY KEY,
            
            -- Basic Info
            material_type TEXT,
            plant TEXT,
            material_number TEXT,
            plant_id TEXT,
            plant_description TEXT,
            uom TEXT,
            
            -- Raw Description Fields
            material_description TEXT,
            po_text TEXT,
            
            -- ============================================================
            -- SEMANTIC SEARCH Fields
            -- ============================================================
            raw_combine_for_embedding TEXT,
            clean_combine_for_embedding TEXT,
            embedding vector({EMBEDDING_DIM}),
            
            -- ============================================================
            -- KEYWORD SEARCH Fields (BM25)
            -- ============================================================
            raw_combine_for_keyword TEXT,
            clean_combine_for_keyword TEXT,
            tsv tsvector,
            
            -- Extracted Keywords/Codes
            keyword_codes TEXT[],
            keyword_old_codes TEXT[],
            keyword_list TEXT[],
            
            -- Metadata
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        print(f"📦 Creating table {SCHEMA_NAME}.{TABLE_NAME}...")
        cur.execute(create_sql)
        
        # Enable required extensions
        print("🔧 Ensuring extensions are enabled...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
        cur.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm")
        
        # Create indexes
        print("🔍 Creating indexes...")
        
        # A) Vector index for semantic search
        cur.execute(f"""
            DO $$
            BEGIN
                IF NOT EXISTS (
                    SELECT 1 FROM pg_indexes 
                    WHERE schemaname='{SCHEMA_NAME}' 
                    AND tablename='{TABLE_NAME}' 
                    AND indexname='idx_{TABLE_NAME}_embedding'
                ) THEN
                    CREATE INDEX idx_{TABLE_NAME}_embedding 
                    ON {SCHEMA_NAME}.{TABLE_NAME} 
                    USING ivfflat (embedding vector_cosine_ops)
                    WITH (lists = 100);
                END IF;
            END
            $$;
        """)
        
        # B) Full-text search index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_tsv 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin(tsv);
        """)
        
        # C) Trigram index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_pk_trgm 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (pk_plant_matnum gin_trgm_ops);
        """)
        
        # D) Array indexes for keyword matching
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_codes 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_codes);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_list 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_list);
        """)
        
        # E) Basic indexes
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_material_type 
            ON {SCHEMA_NAME}.{TABLE_NAME} (material_type);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_uom 
            ON {SCHEMA_NAME}.{TABLE_NAME} (uom);
        """)
        
        # Create trigger for auto-updating tsv
        print("⚙️  Creating trigger for tsvector auto-update...")
        cur.execute(f"""
            CREATE OR REPLACE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger() 
            RETURNS TRIGGER AS $$
            BEGIN
                NEW.tsv := to_tsvector('simple', COALESCE(NEW.clean_combine_for_keyword, ''));
                NEW.updated_at := CURRENT_TIMESTAMP;
                RETURN NEW;
            END;
            $$ LANGUAGE plpgsql;
            
            DROP TRIGGER IF EXISTS tsv_update_trigger ON {SCHEMA_NAME}.{TABLE_NAME};
            
            CREATE TRIGGER tsv_update_trigger 
            BEFORE INSERT OR UPDATE ON {SCHEMA_NAME}.{TABLE_NAME}
            FOR EACH ROW 
            EXECUTE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger();
        """)
        
        conn.commit()
        print(f"✅ Table {SCHEMA_NAME}.{TABLE_NAME} created successfully!")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error creating table: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table")
print("="*60)
# get_connection()
# create_spare_part_table(drop_if_exists=True)  # Uncomment to run

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TEST: Create Table


In [32]:
# ============================================================
# 2. EMBEDDER CLASS (BGE-M3)
# ============================================================
class BGE_M3_Embedder:
    """
    BGE-M3 Embedding Model Wrapper
    """
    def __init__(self, model_name: str = "BAAI/bge-m3", device: str = None):
        """
        Initialize BGE-M3 model
        
        Args:
            model_name: Model identifier (default: BAAI/bge-m3)
            device: 'cuda', 'cpu', or None (auto-detect)
        """
        print(f"🔄 Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name, device=device)
        self.dim = 1024  # bge-m3 output dimension
        print(f"✅ Model loaded! Dimension: {self.dim}")
    
    def embed_texts(
        self, 
        texts: List[str], 
        batch_size: int = 32,
        show_progress: bool = True
    ) -> List[List[float]]:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            batch_size: Batch size for encoding
            show_progress: Show progress bar
            
        Returns:
            List of embedding vectors (each is list of floats)
        """
        if not texts:
            return []
        
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            normalize_embeddings=True  # Recommended for bge-m3
        )
        
        return [emb.tolist() for emb in embeddings]
    
    def embed_single(self, text: str) -> List[float]:
        """Embed a single text (convenience method)"""
        return self.embed_texts([text], show_progress=False)[0]

# Test
print("="*60)
print("TEST: BGE-M3 Embedder")
print("="*60)
# embedder = BGE_M3_Embedder()  # Uncomment to test
# test_emb = embedder.embed_single("BEARING BALL TEST")
# print(f"Embedding dim: {len(test_emb)}")
# print(f"First 5 values: {test_emb[:5]}")

TEST: BGE-M3 Embedder


In [33]:
# ============================================================
# 3. INSERT DATA WITH EMBEDDINGS
# ============================================================
def insert_spare_parts(
    df: pd.DataFrame, 
    embedder: BGE_M3_Embedder, 
    batch_size: int = 128,
    start_from: int = 0
):
    """
    Insert spare part data into database with embeddings
    
    Required DataFrame columns:
        - pk_plant_matnum (PRIMARY KEY)
        - material_type, plant, material_number, plant_id, plant_description, uom
        - material_description, po_text
        - raw_combine_for_embedding, clean_combine_for_embedding
        - raw_combine_for_keyword, clean_combine_for_keyword
        - keyword_codes (list), keyword_old_codes (list), keyword_list (list)
    
    Args:
        df: DataFrame with spare part data
        embedder: BGE_M3_Embedder instance
        batch_size: Number of records to process per batch
        start_from: Skip first N records (for resuming)
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    # Normalize column names to lowercase
    df = df.copy()
    df.columns = df.columns.str.lower()
    
    # Required columns check
    required_cols = [
        'pk_plant_matnum', 'material_type', 'plant', 'material_number', 
        'plantid', 'plant_description', 'uom', 'material_description', 'text',
        'raw_combine_for_embedding', 'clean_combine_for_embedding',
        'raw_combine_for_keyword', 'clean_combine_for_keyword',
        'keyword_codes', 'keyword_old_codes', 'keyword_list'
    ]
    
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    
    # Prepare data
    df_to_insert = df[required_cols].copy()
    total_rows = len(df_to_insert)
    
    if start_from > 0:
        df_to_insert = df_to_insert.iloc[start_from:]
        print(f"⏭️  Skipping first {start_from} records...")
    
    print(f"📊 Total records to insert: {len(df_to_insert)}")
    
    insert_sql = f"""
        INSERT INTO {SCHEMA_NAME}.{TABLE_NAME} (
            pk_plant_matnum, material_type, plant, material_number,
            plant_id, plant_description, uom, material_description, po_text,
            raw_combine_for_embedding, clean_combine_for_embedding, embedding,
            raw_combine_for_keyword, clean_combine_for_keyword,
            keyword_codes, keyword_old_codes, keyword_list
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (pk_plant_matnum) 
        DO UPDATE SET
            material_type = EXCLUDED.material_type,
            material_description = EXCLUDED.material_description,
            clean_combine_for_embedding = EXCLUDED.clean_combine_for_embedding,
            clean_combine_for_keyword = EXCLUDED.clean_combine_for_keyword,
            embedding = EXCLUDED.embedding,
            keyword_codes = EXCLUDED.keyword_codes,
            keyword_list = EXCLUDED.keyword_list,
            updated_at = CURRENT_TIMESTAMP
    """
    
    try:
        inserted = 0
        failed = 0
        
        # Process in batches
        for i in tqdm(range(0, len(df_to_insert), batch_size), desc="Inserting batches"):
            batch_df = df_to_insert.iloc[i:i+batch_size]
            
            # Generate embeddings for this batch
            texts_to_embed = batch_df['clean_combine_for_embedding'].fillna("").tolist()
            embeddings = embedder.embed_texts(texts_to_embed, show_progress=False)
            
            # Prepare batch data
            batch_data = []
            for idx, (_, row) in enumerate(batch_df.iterrows()):
                # Handle column name variations
                plant_id = row.get('plant_id') or row.get('plantid', '')
                po_text = row.get('po_text') or row.get('text', '')
                
                batch_data.append((
                    row['pk_plant_matnum'],
                    row['material_type'],
                    row['plant'],
                    row['material_number'],
                    plant_id,
                    row['plant_description'],
                    row['uom'],
                    row['material_description'],
                    po_text,
                    row['raw_combine_for_embedding'],
                    row['clean_combine_for_embedding'],
                    embeddings[idx],  # embedding vector
                    row['raw_combine_for_keyword'],
                    row['clean_combine_for_keyword'],
                    row['keyword_codes'] if isinstance(row['keyword_codes'], list) else [],
                    row['keyword_old_codes'] if isinstance(row['keyword_old_codes'], list) else [],
                    row['keyword_list'] if isinstance(row['keyword_list'], list) else []
                ))
            
            # Execute batch insert
            try:
                psycopg2.extras.execute_batch(cur, insert_sql, batch_data, page_size=batch_size)
                conn.commit()
                inserted += len(batch_data)
            except Exception as e:
                conn.rollback()
                failed += len(batch_data)
                print(f"\n❌ Batch {i//batch_size + 1} failed: {e}")
                continue
        
        print(f"\n✅ Insertion complete!")
        print(f"   Inserted: {inserted} records")
        if failed > 0:
            print(f"   Failed: {failed} records")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error during insertion: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table & Insert Data")
print("="*60)

import time

# # # Time: Create table
# print("\n🏗️  Creating table...")
# start_time_table = time.time()
# create_spare_part_table(drop_if_exists=True)
# time_table = time.time() - start_time_table
# print(f"✅ Table created in {time_table:.2f} seconds\n")

# # Time: Insert data with embeddings
# print("📥 Inserting data with embeddings...")
# start_time_insert = time.time()
# embedder = BGE_M3_Embedder()
# insert_spare_parts(sample_500, embedder, batch_size=32)
# time_insert = time.time() - start_time_insert
# print(f"✅ Data inserted in {time_insert:.2f} seconds\n")

# print(f"⏱️  Total time: {time_table + time_insert:.2f} seconds")

TEST: Create Table & Insert Data


## Vector Search

In [34]:
# ============================================================
# 4A. VECTOR SEARCH QUERY (SEMANTIC ONLY)
# ============================================================

def vector_search(
    query_text: str,
    embedder: BGE_M3_Embedder,
    top_k_semantic: int = 100,
    limit: int = 50,
    uom_filter: Optional[str] = None,
    material_type_filter: Optional[str] = None
) -> pd.DataFrame:
    """
    Perform pure vector (semantic) search using embeddings only

    Args:
        query_text: Search query text (for embedding)
        embedder: BGE_M3_Embedder instance
        top_k_semantic: Top K results from vector search
        limit: Final result limit
        uom_filter: Optional UOM filter (e.g., 'PC')
        material_type_filter: Optional material type filter

    Returns:
        DataFrame with search results and semantic scores
    """

    # ------------------------------------------------------------
    # 1) Generate query embedding
    # ------------------------------------------------------------
    query_embedding = embedder.embed_single(query_text)

    conn = get_connection()
    cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

    try:
        # ------------------------------------------------------------
        # 2) Build optional filters
        # ------------------------------------------------------------
        filter_clause = ""
        if uom_filter:
            filter_clause += f" AND uom = '{uom_filter}'"
        if material_type_filter:
            filter_clause += f" AND material_type = '{material_type_filter}'"

        # ------------------------------------------------------------
        # 3) Vector search SQL
        # ------------------------------------------------------------
        search_sql = f"""
        WITH semantic_search AS (
            SELECT
                pk_plant_matnum,
                material_description,
                uom,
                (1 - (embedding <=> %s::vector)) AS semantic_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE embedding IS NOT NULL
                {filter_clause}
            ORDER BY embedding <=> %s::vector
            LIMIT {top_k_semantic}
        ),

        normalized_scores AS (
            SELECT
                pk_plant_matnum,
                material_description,
                uom,
                semantic_score,
                CASE
                    WHEN semantic_score > 0
                        THEN semantic_score / MAX(semantic_score) OVER ()
                    ELSE 0
                END AS semantic_norm
            FROM semantic_search
        )

        SELECT
            pk_plant_matnum,
            material_description,
            uom,
            semantic_norm AS final_score,
            ROUND(semantic_norm::numeric, 3) AS semantic_score
        FROM normalized_scores
        ORDER BY final_score DESC
        LIMIT {limit};
        """

        # ------------------------------------------------------------
        # 4) Execute query
        # ------------------------------------------------------------
        cur.execute(search_sql, (
            query_embedding,
            query_embedding
        ))

        results = cur.fetchall()
        df_results = pd.DataFrame(results)
        return df_results

    except Exception as e:
        print(f"❌ Vector search error: {e}")
        raise

    finally:
        cur.close()
        conn.close()


### Evaluate vector search

In [35]:
# ============================================================
# EVALUATION: VECTOR SEARCH QUALITY
# ============================================================

def evaluate_vector_search_quality(
    edges_df,
    sample_df,
    embedder,
    top_k=30
):
    """
    ประเมินคุณภาพของ vector (semantic) search
    โดยเช็คว่าเฉลยอยู่ที่ rank เท่าไหร่

    Args:
        edges_df: DataFrame ที่มี Key_mat_1, Key_mat_2 (คู่ที่เป็นเฉลย)
        sample_df: DataFrame ที่มีข้อมูล material ทั้งหมด (sample_500)
        embedder: BGE_M3_Embedder instance
        top_k: จำนวน candidates สูงสุดที่จะค้นหา

    Returns:
        DataFrame สรุปผลการประเมิน
    """

    # ------------------------------------------------------------
    # 1) Build answer map (pk -> list of correct answers)
    # ------------------------------------------------------------
    answer_map = {}
    for _, row in edges_df.iterrows():
        key1 = row['Key_mat_1']
        key2 = row['Key_mat_2']

        if pd.notna(key1) and pd.notna(key2):
            answer_map.setdefault(key1, []).append(key2)
            answer_map.setdefault(key2, []).append(key1)

    print(f"📊 Total PKs with answers: {len(answer_map)}")
    print(f"🔍 Starting VECTOR search evaluation")
    print(f"⏱️  This may take a few minutes...\n")

    results = []

    # ------------------------------------------------------------
    # 2) Helper: case-insensitive column getter
    # ------------------------------------------------------------
    def get_col(rec, col_name):
        col_lower = col_name.lower()
        for col in rec.index:
            if col.lower() == col_lower:
                return rec[col]
        return ''

    # ------------------------------------------------------------
    # 3) Evaluate each PK
    # ------------------------------------------------------------
    for pk, answers in tqdm(answer_map.items(), desc="Evaluating PKs (Vector Search)"):
        record_rows = sample_df[sample_df['pk_plant_matnum'] == pk]

        if len(record_rows) == 0:
            continue

        record = record_rows.iloc[0]

        query_text = get_col(record, 'clean_combine_for_embedding')
        material_desc = get_col(record, 'material_description')

        if not query_text or pd.isna(query_text):
            continue

        try:
            # ----------------------------------------------------
            # Vector search
            # ----------------------------------------------------
            candidates = vector_search(
                query_text=query_text,
                embedder=embedder,
                limit=top_k
            )

            candidate_pks = candidates['pk_plant_matnum'].tolist()

            # ----------------------------------------------------
            # Find ranks of correct answers
            # ----------------------------------------------------
            found_ranks = []
            for answer_pk in answers:
                if answer_pk in candidate_pks:
                    rank = candidate_pks.index(answer_pk) + 1
                    found_ranks.append(rank)

            if found_ranks:
                best_rank = min(found_ranks)
                status = "✅ Found"
            else:
                best_rank = None
                status = "❌ Not Found"

            results.append({
                'pk': pk,
                'material_description': material_desc if material_desc else '',
                'num_answers': len(answers),
                'answers': ', '.join(answers),
                'found_ranks': ', '.join(map(str, found_ranks)) if found_ranks else 'N/A',
                'best_rank': best_rank,
                'status': status
            })

        except Exception as e:
            print(f"\n⚠️  Error processing {pk}: {e}")
            continue

    # ------------------------------------------------------------
    # 4) Results DataFrame
    # ------------------------------------------------------------
    results_df = pd.DataFrame(results)
    return results_df


In [36]:
print("="*80)
print("EVALUATION: Vector Search Quality Assessment")
print("="*80)

import time
start_time_eval = time.time()

embedder = BGE_M3_Embedder()

evaluation_results_vec = evaluate_vector_search_quality(
    edges_df=sample100,
    sample_df=sample_500,
    embedder=embedder,
    top_k=50
)

time_eval = time.time() - start_time_eval

print(f"\n✅ Evaluation completed in {time_eval:.2f} seconds")
print(evaluation_results_vec.head())
# print(f"⚡ Average time per query: {time_eval/len(evaluation_results_vec):.3f} seconds\n")


EVALUATION: Vector Search Quality Assessment
🔄 Loading embedding model: BAAI/bge-m3...
✅ Model loaded! Dimension: 1024
📊 Total PKs with answers: 163
🔍 Starting VECTOR search evaluation
⏱️  This may take a few minutes...



Evaluating PKs (Vector Search): 100%|██████████| 163/163 [00:11<00:00, 14.58it/s]


✅ Evaluation completed in 18.05 seconds
                    pk                      material_description  num_answers  \
0  7560_75060602310346  BEARING,ANGULAR CONTACT,3310 ATN9  [VMI]            1   
1  7511_75060610300414      BEARING,ANGULAR CONTACT,3310 ATN9(F)            1   
2  7560_75060602411346    BEARING,ANGULAR CONTACT,3311 ANR   [F]            1   
3  7511_75060610310502       BEARING,ANGULAR CONTACT,3311 ANR(F)            1   
4  7812_78060606306581  BEARING,ANGULAR CONTACT,7306 BECBM [VMI]            1   

               answers found_ranks  best_rank   status  
0  7511_75060610300414           3        3.0  ✅ Found  
1  7560_75060602310346           4        4.0  ✅ Found  
2  7511_75060610310502           4        4.0  ✅ Found  
3  7560_75060602411346           3        3.0  ✅ Found  
4  7511_75060610805312           2        2.0  ✅ Found  


In [37]:
# ============================================================
# SUMMARY STATISTICS: VECTOR SEARCH
# ============================================================

print("\n" + "="*80)
print("📈 SUMMARY STATISTICS (VECTOR SEARCH)")
print("="*80)

total = len(evaluation_results_vec)
found = len(evaluation_results_vec[evaluation_results_vec['status'] == '✅ Found'])
not_found = total - found

print(f"Total PKs evaluated: {total}")
print(f"Found in candidates: {found} ({found/total*100:.1f}%)")
print(f"Not found: {not_found} ({not_found/total*100:.1f}%)")

if found > 0:
    found_df = evaluation_results_vec[
        evaluation_results_vec['status'] == '✅ Found'
    ]

    print(f"\nRank Statistics (for found answers):")
    print(f"  Mean Rank: {found_df['best_rank'].mean():.2f}")
    print(f"  Median Rank: {found_df['best_rank'].median():.1f}")
    print(f"  Best Rank: {found_df['best_rank'].min()}")
    print(f"  Worst Rank: {found_df['best_rank'].max()}")

    total_found = len(found_df)

    top_1 = len(found_df[found_df['best_rank'] == 1])
    top_2_3 = len(found_df[(found_df['best_rank'] >= 2) & (found_df['best_rank'] <= 3)])
    top_4_5 = len(found_df[(found_df['best_rank'] >= 4) & (found_df['best_rank'] <= 5)])
    top_6_10 = len(found_df[(found_df['best_rank'] >= 6) & (found_df['best_rank'] <= 10)])
    top_11_20 = len(found_df[(found_df['best_rank'] >= 11) & (found_df['best_rank'] <= 20)])
    top_21_25 = len(found_df[(found_df['best_rank'] >= 21) & (found_df['best_rank'] <= 25)])
    top_26_30 = len(found_df[(found_df['best_rank'] >= 26) & (found_df['best_rank'] <= 30)])
    top_31_40 = len(found_df[(found_df['best_rank'] >= 31) & (found_df['best_rank'] <= 40)])
    over_40 = len(found_df[found_df['best_rank'] > 40])

    print(f"\n📊 Rank Distribution:")
    print(f"  Top 1: {top_1}")
    print(f"  Rank 2–3: {top_2_3}")
    print(f"  Rank 4–5: {top_4_5}")
    print(f"  Rank 6–10: {top_6_10}")
    print(f"  Rank 11–20: {top_11_20}")
    print(f"  Rank 21–25: {top_21_25}")
    print(f"  Rank 26–30: {top_26_30}")
    print(f"  Rank 31–40: {top_31_40}")
    print(f"  Over 40: {over_40}")

    print(
        f"\n✅ Total counted: "
        f"{top_1 + top_2_3 + top_4_5 + top_6_10 + top_11_20 + top_21_25 + top_26_30 + top_31_40 + over_40}"
    )
    print(f"📦 Total rows: {total_found}")

# ============================================================
# CASES WHERE ANSWER NOT FOUND
# ============================================================

print("\n" + "="*80)
print("❌ CASES WHERE ANSWER NOT FOUND (VECTOR SEARCH)")
print("="*80)

not_found_df = evaluation_results_vec[
    evaluation_results_vec['status'] == '❌ Not Found'
]

if len(not_found_df) > 0:
    print(
        not_found_df[['pk', 'material_description', 'answers']]
        .to_string(index=False)
    )
else:
    print("🎉 All answers were found in candidates!")



📈 SUMMARY STATISTICS (VECTOR SEARCH)
Total PKs evaluated: 163
Found in candidates: 159 (97.5%)
Not found: 4 (2.5%)

Rank Statistics (for found answers):
  Mean Rank: 7.48
  Median Rank: 3.0
  Best Rank: 2.0
  Worst Rank: 45.0

📊 Rank Distribution:
  Top 1: 0
  Rank 2–3: 81
  Rank 4–5: 19
  Rank 6–10: 25
  Rank 11–20: 22
  Rank 21–25: 2
  Rank 26–30: 2
  Rank 31–40: 3
  Over 40: 5

✅ Total counted: 159
📦 Total rows: 159

❌ CASES WHERE ANSWER NOT FOUND (VECTOR SEARCH)
                 pk                    material_description            answers
7511_75060621856000       BEARING,DEEP GROOVE,608-2Z,SKF(F) PO11_PO06131330604
7560_75060625503999       BEARING,DEEP GROOVE,6303,SKF  [5] PO21_PO06131330912
7560_75060624704999 BEARING,DEEP GROOVE,6204-2Z/C3,SKF  [F] PO21_PO06131330930
7813_78060617605000   BEARING,DEEP GROOVE,6303-2Z/C3,SKF(F) PO21_PO06131330940


## Prepare data for LLM-Grouping

In [47]:
# ============================================================
# 1. สร้าง test_results สำหรับ LLM-grouping
# ============================================================
"""
test_results จะประกอบด้วย 2 ชุด:
- data ที่มีคำตอบ (15 records) - ต้องมีคำตอบมากกว่า 1 ใน candidates
- data ที่ไม่มีคำตอบ (3 records)
"""

# กำหนดจำนวน records
NUM_WITH_ANSWER = 15
NUM_WITHOUT_ANSWER = 3

# สร้าง answer_map จาก edges (sample100)
answer_map = {}
for _, row in sample100.iterrows():
    key1 = row['Key_mat_1']
    key2 = row['Key_mat_2']
    
    if pd.notna(key1) and pd.notna(key2):
        answer_map.setdefault(key1, []).append(key2)
        answer_map.setdefault(key2, []).append(key1)

print(f"📊 Total PKs with answers: {len(answer_map)}")

# Filter: เอาเฉพาะ PK ที่มี answer มากกว่า 1 และทุกตัวอยู่ใน mm_keys
valid_pks_with_answer = []
for pk, answers in answer_map.items():
    if len(answers) >= 1:  # มี answer อย่างน้อย 1 ตัว
        # เช็คว่า pk และ answers ทุกตัวอยู่ใน mm_keys
        if pk in mm_keys and all(ans in mm_keys for ans in answers):
            valid_pks_with_answer.append(pk)

print(f"✅ Valid PKs with answers (in mm_keys): {len(valid_pks_with_answer)}")

# สุ่ม PKs ที่มีคำตอบ
import random
random.seed(42)
selected_pks_with_answer = random.sample(valid_pks_with_answer, min(NUM_WITH_ANSWER, len(valid_pks_with_answer)))

# ตรวจสอบ duplicate
print(f"Selected PKs with answer: {len(selected_pks_with_answer)}")
print(f"Unique PKs: {len(set(selected_pks_with_answer))}")
if len(selected_pks_with_answer) != len(set(selected_pks_with_answer)):
    print("⚠️ WARNING: Found duplicate PKs!")

# หา PKs ที่ไม่มีคำตอบ (ไม่อยู่ใน answer_map แต่อยู่ใน mm_keys)
pks_without_answer = list(set(sample_500['pk_plant_matnum'].tolist()) - set(answer_map.keys()))
pks_without_answer = [pk for pk in pks_without_answer if pk in mm_keys]

print(f"📋 PKs without answer: {len(pks_without_answer)}")

# สุ่ม PKs ที่ไม่มีคำตอบ
selected_pks_without_answer = random.sample(pks_without_answer, min(NUM_WITHOUT_ANSWER, len(pks_without_answer)))

# สร้าง test_results
test_results = []
case_number = 1

# กรณีที่มีคำตอบ
for query_pk in selected_pks_with_answer:
    record = sample_500[sample_500['pk_plant_matnum'] == query_pk].iloc[0]
    query_description = record.get('Material_Description', '')  # แก้เป็น Material_Description (ตัวใหญ่)
    query_full_text = record.get('clean_combine_for_keyword', query_description)
    
    # ดึง candidates จาก hybrid search (หรือสร้างจาก sample_500)
    # สำหรับตัวอย่างนี้ เราจะใช้ material ทั้งหมดยกเว้นตัวเอง
    candidates_df = sample_500[sample_500['pk_plant_matnum'] != query_pk].copy()
    
    # เพิ่ม dummy score
    candidates_df['final_score'] = 0.5
    
    # Rename columns ให้ตรงกับที่ function ต้องการ (lowercase)
    candidates_for_llm = candidates_df[['pk_plant_matnum', 'material_description', 'clean_combine_for_keyword', 'final_score']].copy()
    # candidates_for_llm = candidates_for_llm.rename(columns={'material_description': 'material_description'})
    
    test_result = {
        'case_number': case_number,
        'query_pk': query_pk,
        'query_description': query_description,
        'query_full_text': query_full_text,
        'has_answer': True,
        'answer_pk': answer_map[query_pk],  # list of answers
        'answer_in_candidates': True,
        'candidates': candidates_for_llm.to_dict('records')
    }
    
    test_results.append(test_result)
    case_number += 1

# กรณีที่ไม่มีคำตอบ
for query_pk in selected_pks_without_answer:
    record = sample_500[sample_500['pk_plant_matnum'] == query_pk].iloc[0]
    query_description = record.get('Material_Description', '')  # แก้เป็น Material_Description (ตัวใหญ่)
    query_full_text = record.get('clean_combine_for_keyword', query_description)
    
    # ดึง candidates
    candidates_df = sample_500[sample_500['pk_plant_matnum'] != query_pk].copy()
    candidates_df['final_score'] = 0.5
    
    # Rename columns ให้ตรงกับที่ function ต้องการ (lowercase)
    candidates_for_llm = candidates_df[['pk_plant_matnum', 'material_description', 'clean_combine_for_keyword', 'final_score']].copy()
    candidates_for_llm = candidates_for_llm.rename(columns={'material_description': 'material_description'})
    
    test_result = {
        'case_number': case_number,
        'query_pk': query_pk,
        'query_description': query_description,
        'query_full_text': query_full_text,
        'has_answer': False,
        'answer_pk': None,
        'answer_in_candidates': False,
        'candidates': candidates_for_llm.to_dict('records')
    }
    
    test_results.append(test_result)
    case_number += 1

print(f"\n{'='*60}")
print(f"✅ test_results created:")
print(f"   - Total cases: {len(test_results)}")
print(f"   - With answer: {len([t for t in test_results if t['has_answer']])}")
print(f"   - Without answer: {len([t for t in test_results if not t['has_answer']])}")
print(f"{'='*60}")

# ตรวจสอบ duplicate case_number
case_numbers = [t['case_number'] for t in test_results]
print(f"\nCase numbers: {case_numbers}")
print(f"Unique case numbers: {len(set(case_numbers))}")
if len(case_numbers) != len(set(case_numbers)):
    print("⚠️ WARNING: Found duplicate case numbers!")
    from collections import Counter
    duplicates = [num for num, count in Counter(case_numbers).items() if count > 1]
    print(f"   Duplicate case numbers: {duplicates}")

# ตรวจสอบ duplicate query_pk
query_pks = [t['query_pk'] for t in test_results]
print(f"\nUnique query PKs: {len(set(query_pks))}")
if len(query_pks) != len(set(query_pks)):
    print("⚠️ WARNING: Found duplicate query PKs!")
    from collections import Counter
    duplicates = [pk for pk, count in Counter(query_pks).items() if count > 1]
    print(f"   Duplicate PKs: {duplicates}")

# แสดงตัวอย่าง
print("\n📋 Sample test_results:")
for i, test in enumerate(test_results[:21], 1):
    print(f"\nCase {test['case_number']}:")
    print(f"  Query PK: {test['query_pk'][:40]}...")
    print(f"  Query Description: {test['query_description']}")
    print(f"  Full Text: {test['query_full_text']}")
    print(f"  Has answer: {test['has_answer']}")
    print(f"  Candidates: {len(test['candidates'])} items")
    if test['has_answer']:
        answers = test['answer_pk'] if isinstance(test['answer_pk'], list) else [test['answer_pk']]
        print(f"  Answer PKs: {len(answers)} items - {answers[:3]}")

📊 Total PKs with answers: 163
✅ Valid PKs with answers (in mm_keys): 163
Selected PKs with answer: 15
Unique PKs: 15
📋 PKs without answer: 400

✅ test_results created:
   - Total cases: 18
   - With answer: 15
   - Without answer: 3

Case numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Unique case numbers: 18

Unique query PKs: 18

📋 Sample test_results:

Case 1:
  Query PK: K011_K0060615920103...
  Query Description: 
  Full Text: bearing , cylindrical nu 1015 ml bearing , internal clearance nu 1015 ml type cylindrical , mfr part ident no.4045269 , short name bearing cylinder nu 1015 ml iso nu 1015 ml mfr name skf application stock preparation , remark bearing cylinder roller ident no.4045269 d 75/115 x 20 item 98 for stock preparation snp pc
  Has answer: True
  Candidates: 562 items
  Answer PKs: 1 items - ['7511_75060615969205']

Case 2:
  Query PK: 7812_78060612308552...
  Query Description: 
  Full Text: bearing , cylindrical , nu 308 ecp/c3 [f] bearing ,

In [39]:
print(test_results[1])

{'case_number': 2, 'query_pk': '7812_78060612308552', 'query_description': '', 'query_full_text': 'bearing , cylindrical , nu 308 ecp/c3 [f] bearing , bore type cylindrical , internal clearance c3 , type cylindrical , mfr part nu 308 ecp/c3 , iso nu308 , seal/shield type no seal , mfr name skf , cage material metal , application lime mud washer pump plant recausticizing pc', 'has_answer': True, 'answer_pk': ['7511_75060612500369'], 'answer_in_candidates': True, 'candidates': [{'pk_plant_matnum': '7560_75060602310346', 'material_description': 'BEARING,ANGULAR CONTACT,3310 ATN9  [VMI]', 'clean_combine_for_keyword': 'bearing , angular contact , 3310 atn9 [vmi] bearing , bore type cylindrical , internal clearance normal clearance , seal/shield type no seal , type angular contact , mfr part 3310 atn9 , iso 3310 , mfr name skf , application gorman-rupp pump model t104a24-b , u6b60-b (tkic#6 , 7) pc', 'final_score': 0.5}, {'pk_plant_matnum': '7560_75060602411346', 'material_description': 'BEA

In [40]:
# ============================================================
# 2. สร้าง selected_examples สำหรับ few-shot learning
# ============================================================
"""
selected_examples จะใช้เป็นตัวอย่างให้ LLM เข้าใจว่า
material PKs ที่เป็นของชนิดเดียวกันมีลักษณะอย่างไร
"""

# จำนวนตัวอย่างที่ต้องการ
NUM_EXAMPLES = 5

# สุ่มตัวอย่างจาก edges (sample100)
random.seed(42)
example_edges = sample100.sample(n=min(NUM_EXAMPLES, len(sample100)))

selected_examples = []

for idx, row in example_edges.iterrows():
    key1 = row['Key_mat_1']
    key2 = row['Key_mat_2']
    
    # ดึงข้อมูลของ key1 (query)
    record1 = sample_500[sample_500['pk_plant_matnum'] == key1]
    if len(record1) == 0:
        continue
    record1 = record1.iloc[0]
    
    # ดึงข้อมูลของ key2 (answer)
    record2 = sample_500[sample_500['pk_plant_matnum'] == key2]
    if len(record2) == 0:
        continue
    record2 = record2.iloc[0]
    
    # สร้าง example ตาม format ที่ function ต้องการ
    example = {
        'query_pk': key1,
        'query_description': record1.get('material_description', ''),
        'clean_combine_for_keyword': record1.get('clean_combine_for_keyword', ''),
        'answer_pk': key2,
        'answer_description': record2.get('material_description', ''),
        'answer_full_text': record2.get('clean_combine_for_keyword', '')
    }
    
    selected_examples.append(example)

print(f"\n{'='*60}")
print(f"✅ selected_examples created:")
print(f"   - Total examples: {len(selected_examples)}")
print(f"{'='*60}")

# แสดงตัวอย่าง
print("\n📋 Sample selected_examples:")
for i, example in enumerate(selected_examples[:2], 1):
    print(f"\nExample {i}:")
    print(f"  Query PK: {example['query_pk'][:40]}...")
    print(f"  Query Desc: {example['query_description'][:60]}...")
    print(f"  Answer PK: {example['answer_pk'][:40]}...")
    print(f"  Answer Desc: {example['answer_description'][:60]}...")


✅ selected_examples created:
   - Total examples: 5

📋 Sample selected_examples:

Example 1:
  Query PK: 7560_75060625509999...
  Query Desc: BEARING,DEEP GROOVE,6309,SKF  [F]...
  Answer PK: 7511_75060621641000...
  Answer Desc: BEARING,DEEP GROOVE,6309,SKF(F)...

Example 2:
  Query PK: 7812_78060612308552...
  Query Desc: BEARING,CYLINDRICAL,NU 308 ECP/C3 [F]...
  Answer PK: 7511_75060612500369...
  Answer Desc: BEARING,CYLINDRICAL,NU 308 ECP/C3...


In [44]:
# ============================================================
# Hard-code Few-shot Examples จาก selected_examples
# ============================================================
"""
เลือก 3 examples ที่น่าสนใจจาก selected_examples:
- Example index 0, 1, 2 (หรือเลือกตาม case ที่ต้องการ)
"""

print("📋 Inspecting selected_examples to create hard-coded few_shot_examples...\n")

# แสดงข้อมูล selected_examples ทั้งหมดเพื่อเลือก
for i, ex in enumerate(selected_examples):
    print(f"Example {i}:")
    print(f"  Query PK: {ex['query_pk']}")
    print(f"  Answer PK: {ex['answer_pk']}")
    print(f"  Query Desc: {ex['query_description'][:80]}")
    print(f"  Answer Desc: {ex['answer_description'][:80]}")
    print()

# เลือก 3 examples ที่ดีที่สุด
selected_indices = [0, 1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

# สร้าง hard-coded few_shot_examples
few_shot_examples = []

for idx in selected_indices:
    if idx < len(selected_examples):
        ex = selected_examples[idx]
        few_shot_examples.append({
            'query_pk': ex['query_pk'],
            'query_description': ex['query_description'],
            'clean_combine_for_keyword': ex['clean_combine_for_keyword'],
            'answer_pk': ex['answer_pk'],
            'answer_description': ex['answer_description'],
            'answer_full_text': ex.get(
                'answer_full_text',
                ex.get('clean_combine_for_keyword', '')
            )
        })

print(f"\n{'='*60}")
print("✅ few_shot_examples created (hard-coded from selected_examples):")
print(f"   - Total examples: {len(few_shot_examples)}")
print(f"{'='*60}")

# แสดงผล few_shot_examples ที่สร้างแล้ว
print("\n📋 Few-shot Examples for LLM:")
for i, example in enumerate(few_shot_examples, 1):
    print(f"\nExample {i}:")
    print(f"  Query PK: {example['query_pk']}")
    print(f"  Query: {example['query_description'][:70]}")
    print(f"  PK Description: {example['clean_combine_for_keyword']}")
    print(f"  Answer PK: {example['answer_pk']}")
    print(f"  Answer PK Description: {example['answer_full_text']}")
    print(f"  Reason: Same spare part with matching specifications")


📋 Inspecting selected_examples to create hard-coded few_shot_examples...

Example 0:
  Query PK: 7560_75060625509999
  Answer PK: 7511_75060621641000
  Query Desc: BEARING,DEEP GROOVE,6309,SKF  [F]
  Answer Desc: BEARING,DEEP GROOVE,6309,SKF(F)

Example 1:
  Query PK: 7812_78060612308552
  Answer PK: 7511_75060612500369
  Query Desc: BEARING,CYLINDRICAL,NU 308 ECP/C3 [F]
  Answer Desc: BEARING,CYLINDRICAL,NU 308 ECP/C3

Example 2:
  Query PK: 7560_75060625503999
  Answer PK: PO21_PO06131330912
  Query Desc: BEARING,DEEP GROOVE,6303,SKF  [5]
  Answer Desc: BEARING,DEEPGROOVE,6303

Example 3:
  Query PK: PO11_PO06131335063
  Answer PK: PO21_PO06131335390
  Query Desc: BEARING,SPHERICAL ROLLER,22316 E (S)
  Answer Desc: BEARING,SPHERICAL,22316E

Example 4:
  Query PK: 7560_75060626114999
  Answer PK: 7511_75060621630771
  Query Desc: BEARING,DEEP GROOVE,6314-2Z/C3,SKF  [F]
  Answer Desc: BEARING,DEEP GROOVE,6314-2Z/C3,SKF(F)


✅ few_shot_examples created (hard-coded from selected_examples

### ✅ LLM Material Grouping - UPDATED VERSION

**สิ่งที่ปรับปรุง:**

1. **Output Schema ที่เรียบง่าย** - เอาเฉพาะข้อมูลที่จำเป็น:
   - `status`: "match" หรือ "unmatch" 
   - `matched_pks`: list ของ PKs ที่เป็น duplicate (ไม่รวม query_pk เอง)
   - `summary`: สรุปสั้นๆ
   
2. **Prompt ที่ชัดเจนขึ้น** - บอก LLM ให้ return เฉพาะ PKs ที่ซ้ำกัน ไม่ต้องส่ง analysis แต่ละตัว

3. **Validation ที่ถูกต้อง** - ใช้ PKs จริงจาก edges.csv:
   - 7560_75060675913234 (ไม่ใช่ 7560_75060670873346)
   - PO11_PO06131335013

In [ ]:
# ============================================================
# IMPROVED LLM Function with Step-by-Step Reasoning
# ============================================================

from pydantic import BaseModel, Field
from typing import List

class MatchedMaterial(BaseModel):
    """Material ที่ตรงกับ query"""
    pk: str = Field(description="PK ของ candidate ที่เป็น duplicate")
    confidence: float = Field(description="ความมั่นใจ 0.0-1.0", ge=0.0, le=1.0)

class ImprovedMaterialGroupingResult(BaseModel):
    """ผลลัพธ์การจัดกลุ่ม materials แบบปรับปรุง"""
    query_pk: str
    status: str  # "match" หรือ "unmatch"
    matched_materials: List[MatchedMaterial] = Field(default_factory=list)
    summary: str

def improved_group_materials_with_llm(
    query_pk: str,
    query_description: str,
    query_full_text: str,
    candidates: pd.DataFrame,
    few_shot_examples: List[dict],
    model: str = "gemini-3-flash-preview",
    min_confidence: float = 0.7
) -> ImprovedMaterialGroupingResult:
    """
    ปรับปรุง LLM grouping ด้วย step-by-step reasoning และ few-shot examples
    """
    
    # เตรียม client
    client = genai.Client(
        vertexai={
            "project": "prj-service-mlops",
            "location": "global"
        }
    )
    
    # เตรียม candidates
    candidate_list = []
    for idx, row in candidates.iterrows():
        candidate_list.append({
            "pk": row['pk_plant_matnum'],
            "description": row['material_description'],
            "full_text": row.get('clean_combine_for_keyword')
        })
    
    # สร้าง Few-Shot Examples section
    # examples_text = ""
    # for i, ex in enumerate(few_shot_examples, 1):
#         examples_text += f"""
# ### Example {i}: Materials that ARE Duplicates

# **Query Material:**
# - PK: {ex['query_pk']}
# - Description: {ex['query_description']}
# - Full Text: {ex['clean_combine_for_keyword']}

# **Candidate Material (DUPLICATE):**
# - PK: {ex['answer_pk']}
# - Description: {ex['answer_description']}
# - Full Text: {ex['clean_combine_for_keyword']}

# ---
# """
    examples_text = """
Example 1:
**Query Material:**
- PK: K011_K0060615920103
- Description: BEARING,CYLINDRICAL NU 1015 ML
- Full Text: bearing , cylindrical nu 1015 ml bearing , internal clearance nu 1015 ml type cylindrical , mfr part ident no.4045269 , short name bearing cylinder nu 1015 ml iso nu 1015 ml mfr name skf application stock preparation , remark bearing cylinder roller ident no.4045269 d 75/115 x 20 item 98 for stock preparation snp pc

**Candidate Material (DUPLICATE) (Answer):**
- PK: 7511_75060615969205
- Description: BEARING,CYLINDRICAL,NU 1015(VMI)
- Full Text: bearing , cylindrical , nu 1015(vmi) bearing , bore type cylindrical , internal clearance normal clearance , type cylindrical , mfr part nu 1015 , short name bearing , seal/shield type no seal , mfr name skf , cage material brass , application minisorter at wp.1 , remark bearing cylindrical roller no.nu 1015 m or j , use for minisorter at wp.1 pc
- Reason: Both records describe the same SKF cylindrical roller bearing NU 1015. They share the identical bearing type and model number, and the query’s “ML” suffix (machined brass cage) is technically equivalent to the candidate’s specification of a brass cage (“NU 1015 m”). The standard ISO dimensions 75×115×20 mm also match exactly, confirming they represent the same physical material.

Example 2:
**Query Material:**
- PK: PO11_PO06131330975
- Description: BEARING,DEEP GROOVE,6012 2Z
- Full Text: bearing , deep groove , 6012 2z bearing , bore type cylindrical , internal clearance normal clearance , seal/shield type 2z , type deep groove , mfr part 6012 2z , remark (mai.) for mount hope size press pmh-001 pc

**Candidate Material (DUPLICATE) (Have 2 Answer):**
- PK: 7511_75060621901500
- Description: BEARING,DEEP GROOVE,6012-2Z,SKF(F)
- Full Text: bearing , deep groove , 6012-2z , skf(f) bearing , bore type cylindrical , internal clearance normal clearance , seal/shield type 2 metal seal , type deep groove , mfr part 6012-2z , short name bearing , iso 6012 , mfr name skf , cage material metal , remark bearing deep groove ball 6012-2z skf or equivalent ) pc

- PK: 7580_75060622612744
- Description: BEARING,DEEP GROOVE,6012-2Z,SKF [F]
- Full Text: bearing , deep groove , 6012-2z , skf [f] bearing , bore type cylindrical , internal clearance normal clearance , seal/shield type 2 metal seal , type deep groove , mfr part 6012-2z , iso 6012 , mfr name skf , cage material metal , application cv#8 bottom slitter#1 , single cutter#2 pc

- Reason: All matched records represent the same deep groove ball bearing with ISO part number 6012-2Z. They share identical core technical specifications, including bearing type, size, normal internal clearance, cylindrical bore, and double metal shields (2Z). The differences between records are limited to descriptive text or application notes, which do not affect the physical identity of the material. Therefore, these records are grouped as the same material.

Example 3:
**Query Material:**
- PK: 7580_75060765160745
- Description: BELT,V,NARROW,5V 1600 [N]
- Full Text: belt , v , narrow , 5v 1600 [n] belt , v , belt section 5 v , type narrow , belt rma 5v 1600 , of strands 1600 , application double cutter gear box no.1 pc

**Candidate Material (DUPLICATE) (Answer):**
- PK: 7511_75060755473000
- Description: BELT,V,NARROW,5V1600
- Full Text: belt , v , narrow , 5v1600 belt , v , short name belt , v , belt section 5 v , type narrow , belt rma 5v1600 , of strands 1600 , application couch pith agitator , remark belt v section 5v-1600 skic = 4 pcs/time tpc = 6 pcs/time sppc = 4 pcs/time for couch pit agitator pm.1 , tpc , sppc pc
- Reason: Both records describe the same narrow V-belt with RMA specification 5V 1600. They share the identical belt section (5V) and length (1600), which uniquely define the physical product. Any differences between records are limited to system or plant registration details and do not affect the material identity.

---
"""


    # สร้าง prompt ใหม่
    prompt = f"""# Advanced Material Deduplication with Step-by-Step Analysis

You are an expert in industrial spare parts identification. Your task is to identify which candidate materials are EXACT DUPLICATES of the query material using systematic analysis.

## Few-Shot Examples (Materials that ARE Duplicates):
{examples_text}

## Current Task:

### Query Material (Find duplicates of this):
- **PK**: {query_pk}
- **Description**: {query_description}
- **Full Text**: {query_full_text}

### Candidate Materials (Analyze each one):
"""
    
    for i, cand in enumerate(candidate_list, 1):
        prompt += f"""
**Candidate #{i}:**
- PK: {cand['pk']}
- Description: {cand['description']}
- Full Text: {cand['full_text']}
"""
    
    prompt += """

## 🔍 Analysis Framework (Follow these steps):

**Step 1: Extract Key Identifiers**
- Part numbers, model codes, manufacturer codes
- Size/dimensions (numbers with units like mm, cm, inch)
- Material type (steel, plastic, rubber, etc.)
- Capacity/rating (voltage, pressure, flow rate)

**Step 2: Compare Technical Specifications**
- Are dimensions identical or equivalent?
- Same material composition?
- Same technical ratings?
- Same functionality?

**Step 3: Evaluate Semantic Similarity**
- Do descriptions refer to the same product?
- Are they used for the same purpose?
- Same application area?

**Step 4: Determine Confidence Level**
- **High (0.9-1.0)**: Identical part numbers AND specs
- **Medium-High (0.8-0.89)**: Same specs, similar codes
- **Medium (0.7-0.79)**: Same specs, different codes
- **Below 0.7**: Uncertain or different materials

**Step 5: Make Final Decision**
- Only include matches with confidence ≥ 0.7
- When uncertain, be conservative (don't match)

## Matching Rules:

**MATCH if:**
- Same part number/model code (even with plant prefix)
- Same specifications (size, material, capacity)
- Different only in: plant code, vendor ID, packaging info

**DO NOT MATCH if:**
- Different part numbers AND different specs
- Different sizes or capacities
- Different materials or types
- One is accessory, another is main product
- Similar but not identical

## Output Format:

For each duplicate found, provide:
1. **pk**: The candidate PK
2. **confidence**: 0.7-1.0 (≥0.7 to include)
3. **reason**: Brief explanation (30-50 words)

Include **step_by_step_reasoning** showing your analysis process.

## Important Notes:

- Be systematic - analyze ALL candidates
- Be conservative - only high confidence matches
- Ignore the first candidate if it's the query itself (same PK)
- Focus on TECHNICAL match, not just keyword similarity
- Different plants can have same material

Analyze carefully and provide your results."""

    # เรียก LLM
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=ImprovedMaterialGroupingResult,
            temperature=0.1,  # เพิ่มเล็กน้อยเพื่อความยืดหยุ่น
            top_p=0.95
        )
    )
    
    # Parse response
    result = ImprovedMaterialGroupingResult.model_validate_json(response.text)
    
    # Filter by confidence threshold
    filtered_matches = [
        m for m in result.matched_materials 
        if m.confidence >= min_confidence
    ]
    
    result.matched_materials = filtered_matches
    if not filtered_matches:
        result.status = "unmatch"
    
    return result

print("✅ Improved LLM function created")
print(f"   Model: gemini-3-flash-preview")
print(f"   Min confidence: 0.7")
print(f"   Features: Step-by-step reasoning + Few-shot examples")

In [ ]:
# ============================================================
# IMPROVED LLM Function (Optimized - No Reasoning)
# ============================================================

from pydantic import BaseModel, Field
from typing import List

class MatchedMaterial(BaseModel):
    """Material ที่ตรงกับ query"""
    pk: str = Field(description="PK ของ candidate ที่เป็น duplicate")
    confidence: float = Field(description="ความมั่นใจ 0.0-1.0", ge=0.0, le=1.0)

class ImprovedMaterialGroupingResult(BaseModel):
    """ผลลัพธ์การจัดกลุ่ม materials แบบปรับปรุง"""
    query_pk: str
    status: str  # "match" หรือ "unmatch"
    matched_materials: List[MatchedMaterial] = Field(default_factory=list)
    summary: str

def improved_group_materials_with_llm(
    query_pk: str,
    query_description: str,
    query_full_text: str,
    candidates: pd.DataFrame,
    few_shot_examples: List[dict],
    model: str = "gemini-3-flash-preview",
    min_confidence: float = 0.7
) -> ImprovedMaterialGroupingResult:
    """
    ปรับปรุง LLM grouping แบบรวดเร็ว (ไม่ต้อง reasoning)
    """
    
    # เตรียม client
    client = genai.Client(
        vertexai={
            "project": "prj-service-mlops",
            "location": "global"
        }
    )
    
    # เตรียม candidates
    candidate_list = []
    for idx, row in candidates.iterrows():
        candidate_list.append({
            "pk": row['pk_plant_matnum'],
            "description": row['material_description'],
            "full_text": row.get('clean_combine_for_keyword')
        })
    
    # สร้าง Few-Shot Examples แบบกระชับ
    examples_text = """
Example 1: SKF Cylindrical Bearing NU 1015
Query PK: K011_K0060615920103 (BEARING,CYLINDRICAL NU 1015 ML)
→ Duplicate: 7511_75060615969205 (BEARING,CYLINDRICAL,NU 1015(VMI))
(Same SKF bearing NU 1015, brass cage, 75×115×20mm)

Example 2: Deep Groove Ball Bearing 6012-2Z  
Query PK: PO11_PO06131330975 (BEARING,DEEP GROOVE,6012 2Z)
→ Duplicates: 7511_75060621901500, 7580_75060622612744
(Same ISO 6012-2Z, double metal shields, SKF)

Example 3: Narrow V-Belt 5V 1600
Query PK: 7580_75060765160745 (BELT,V,NARROW,5V 1600 [N])
→ Duplicate: 7511_75060755473000 (BELT,V,NARROW,5V1600)
(Same RMA 5V 1600, section 5V, length 1600)
"""

    # สร้าง prompt แบบกระชับ
    prompt = f"""# Industrial Spare Parts Deduplication

You are an expert in identifying duplicate spare parts. Find EXACT DUPLICATES of the query material from candidates.

## Examples of Duplicate Materials:
{examples_text}

## Current Task:

### Query Material:
- **PK**: {query_pk}
- **Description**: {query_description}
- **Full Text**: {query_full_text}

### Candidate Materials:
"""
    
    for i, cand in enumerate(candidate_list, 1):
        prompt += f"""
**Candidate #{i}:**
- PK: {cand['pk']}
- Description: {cand['description']}
- Full Text: {cand['full_text']}
"""
    
    prompt += """

## Matching Rules:

**MATCH if:**
- Same part number/model code (ignore plant prefix)
- Same technical specs (size, material, capacity, type)
- Only differ in: plant code, vendor ID, application notes

**DO NOT MATCH if:**
- Different part numbers AND different specs
- Different sizes, capacities, or materials
- One is accessory, another is main product

**Confidence Levels:**
- **0.9-1.0**: Identical part numbers + specs
- **0.8-0.89**: Same specs, similar codes
- **0.7-0.79**: Same specs, different codes
- **Below 0.7**: Don't include

## Instructions:

1. Compare each candidate with query material
2. Match on part numbers, dimensions, technical specs
3. Only include matches with confidence ≥ 0.7
4. Skip if candidate PK = query PK
5. Be conservative - only match if truly identical

Provide results in JSON format with: pk, confidence, and summary."""

    # เรียก LLM
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=ImprovedMaterialGroupingResult,
            temperature=0.1,
            top_p=0.95
        )
    )
    
    # Parse response
    result = ImprovedMaterialGroupingResult.model_validate_json(response.text)
    
    # Filter by confidence threshold
    filtered_matches = [
        m for m in result.matched_materials 
        if m.confidence >= min_confidence
    ]
    
    result.matched_materials = filtered_matches
    if not filtered_matches:
        result.status = "unmatch"
    
    return result

print("✅ Improved LLM function created (Optimized - No Reasoning)")
print(f"   Model: gemini-3-flash-preview")
print(f"   Min confidence: 0.7")
print(f"   Features: Fast matching + Concise few-shot examples")

✅ Improved LLM function created
   Model: gemini-3-flash-preview
   Min confidence: 0.7
   Features: Step-by-step reasoning + Few-shot examples


In [ ]:
# ============================================================
# ทดสอบ Improved LLM กับ Test Cases
# ============================================================

# Clear output (optional - for notebook clarity)
from IPython.display import clear_output
import time

print("\n" + "="*80)
print("🧪 TESTING IMPROVED LLM (gemini-3-flash-preview)")
print("="*80)

# ตรวจสอบ test_results ก่อน
print(f"\n📊 Test Results Summary:")
print(f"   - Total test cases: {len(test_results)}")
print(f"   - Unique case numbers: {len(set([t['case_number'] for t in test_results]))}")
print(f"   - Unique query PKs: {len(set([t['query_pk'] for t in test_results]))}")

# ตรวจสอบ duplicate
if len(test_results) != len(set([t['case_number'] for t in test_results])):
    print("\n⚠️  WARNING: Found duplicate case numbers in test_results!")
    print("   Please re-run cell 58 to regenerate test_results")
    # หยุดการ execute
    raise ValueError("Duplicate case numbers detected. Please re-run cell 58.")

if len(test_results) != len(set([t['query_pk'] for t in test_results])):
    print("\n⚠️  WARNING: Found duplicate query PKs in test_results!")
    print("   Please re-run cell 58 to regenerate test_results")
    # หยุดการ execute
    raise ValueError("Duplicate query PKs detected. Please re-run cell 58.")

print("\n✅ test_results validation passed")
print(f"🚀 Starting LLM evaluation on {len(test_results)} test cases...\n")

# Clear และสร้าง improved_llm_results ใหม่
improved_llm_results = []

start_time = time.time()

for i, test_result in enumerate(test_results, 1):
    # เพิ่ม separator ชัดเจน
    print(f"\n{'─'*80}")
    print(f"[{i}/{len(test_results)}] 🔍 Case {test_result['case_number']}: {test_result['query_pk'][:40]}...")
    print(f"{'─'*80}")
    
    # ดึง query info
    query_pk = test_result['query_pk']
    query_description = test_result['query_description']
    
    query_record = sample_500[sample_500['pk_plant_matnum'] == query_pk].iloc[0]
    query_full_text = query_record.get('clean_combine_for_keyword', query_description)
    
    # แปลง candidates เป็น DataFrame
    candidates_df = pd.DataFrame(test_result['candidates'])
    
    try:
        # ส่งให้ Improved LLM
        print(f"   📤 Sending to improved LLM...")
        
        improved_result = improved_group_materials_with_llm(
            query_pk=query_pk,
            query_description=query_description,
            query_full_text=query_full_text,
            candidates=candidates_df,
            few_shot_examples=selected_examples,
            model="gemini-3-flash-preview",
            min_confidence=0.7
        )
        
        # เก็บผลลัพธ์
        result_dict = {
            "case_number": test_result['case_number'],
            "query_pk": test_result['query_pk'],
            "has_answer": test_result['has_answer'],
            "answer_pk": test_result['answer_pk'],
            "answer_in_candidates": test_result.get('answer_in_candidates'),
            "answer_rank": test_result.get('answer_rank'),
            "llm_response": improved_result,
            "matched_pks": [m.pk for m in improved_result.matched_materials]
        }
        
        # แสดงผลลัพธ์
        if improved_result.status == "match" and improved_result.matched_materials:
            print(f"   ✅ Status: {improved_result.status}")
            print(f"   📋 LLM Found {len(improved_result.matched_materials)} matches:")
            for m in improved_result.matched_materials[:5]:
                print(f"      - {m.pk[:50]}... (conf: {m.confidence:.2f})")
        else:
            print(f"   ❌ Status: {improved_result.status} (no matches)")
        
        # เช็คความถูกต้อง
        if test_result['has_answer']:
            # แสดง actual answers - รองรับทั้ง list และ single value
            answer_pk_raw = test_result['answer_pk']
            if answer_pk_raw is None:
                actual_answers = []
            elif isinstance(answer_pk_raw, list):
                actual_answers = answer_pk_raw
            else:
                actual_answers = [answer_pk_raw]
            
            print(f"\n   🎯 ACTUAL ANSWERS ({len(actual_answers)}):")
            for ans in actual_answers:
                print(f"      - {ans}")
            
            print(f"\n   🤖 LLM ANSWERS ({len(result_dict['matched_pks'])}):")
            if len(result_dict['matched_pks']) > 0:
                for llm_ans in result_dict['matched_pks']:
                    print(f"      - {llm_ans}")
            else:
                print(f"      (No matches found)")
            
            # เช็คว่า LLM หาเจอคำตอบที่ถูกต้องหรือไม่
            found_answers = []
            missed_answers = []
            for ans in actual_answers:
                if ans in result_dict['matched_pks']:
                    found_answers.append(ans)
                else:
                    missed_answers.append(ans)
            
            # คำนวณ precision/recall
            num_found = len(found_answers)
            num_total_actual = len(actual_answers)
            num_total_llm = len(result_dict['matched_pks'])
            
            result_dict['llm_correct'] = num_found > 0
            result_dict['num_found'] = num_found
            result_dict['num_missed'] = len(missed_answers)
            result_dict['num_false_positive'] = num_total_llm - num_found
            
            if num_found > 0:
                print(f"\n   ✅ CORRECT! Found {num_found}/{num_total_actual} answers")
                for ans in found_answers:
                    # หา confidence ของ answer
                    answer_match = next((m for m in improved_result.matched_materials 
                                        if m.pk == ans), None)
                    if answer_match:
                        print(f"      ✓ {ans} (confidence: {answer_match.confidence:.2f})")
                
                if len(missed_answers) > 0:
                    print(f"\n   ⚠️  But MISSED {len(missed_answers)} answers:")
                    for ans in missed_answers:
                        print(f"      ✗ {ans}")
                
                if result_dict['num_false_positive'] > 0:
                    print(f"\n   ⚠️  Found {result_dict['num_false_positive']} FALSE POSITIVES (incorrect matches)")
            else:
                print(f"\n   ❌ MISSED all {num_total_actual} answers:")
                for ans in missed_answers:
                    print(f"      ✗ {ans}")
        else:
            result_dict['llm_correct'] = None
            print(f"\n   ℹ️  This case has NO expected answer (negative example)")
            
            if len(result_dict['matched_pks']) > 0:
                print(f"   ⚠️  But LLM found {len(result_dict['matched_pks'])} matches (FALSE POSITIVES):")
                for llm_ans in result_dict['matched_pks'][:3]:
                    print(f"      - {llm_ans}")
        
        improved_llm_results.append(result_dict)
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        result_dict = {
            "case_number": test_result['case_number'],
            "query_pk": test_result['query_pk'],
            "has_answer": test_result['has_answer'],
            "answer_pk": test_result['answer_pk'],
            "error": str(e)
        }
        improved_llm_results.append(result_dict)
        continue

print(f"\n{'='*80}")
print(f"✅ Improved LLM Complete: {len(improved_llm_results)}/{len(test_results)} cases")
print("="*80)


🧪 TESTING IMPROVED LLM (gemini-3-flash-preview)

📊 Test Results Summary:
   - Total test cases: 18
   - Unique case numbers: 18
   - Unique query PKs: 18

✅ test_results validation passed
🚀 Starting LLM evaluation on 18 test cases...


────────────────────────────────────────────────────────────────────────────────
[1/18] 🔍 Case 1: K011_K0060615920103...
────────────────────────────────────────────────────────────────────────────────
   📤 Sending to improved LLM...
   ❌ Status: success (no matches)

   🎯 ACTUAL ANSWERS (1):
      - 7511_75060615969205

   🤖 LLM ANSWERS (1):
      - 7511_75060615969205

   ✅ CORRECT! Found 1/1 answers
      ✓ 7511_75060615969205 (confidence: 0.95)

────────────────────────────────────────────────────────────────────────────────
[2/18] 🔍 Case 2: 7812_78060612308552...
────────────────────────────────────────────────────────────────────────────────
   📤 Sending to improved LLM...
   ❌ Status: success (no matches)

   🎯 ACTUAL ANSWERS (1):
      - 7511_75

In [62]:
print(result_dict)

{'case_number': 18, 'query_pk': '7560_54060650443573', 'has_answer': False, 'answer_pk': None, 'answer_in_candidates': False, 'answer_rank': None, 'llm_response': ImprovedMaterialGroupingResult(query_pk='7560_54060650443573', status='unmatch', matched_materials=[], step_by_step_reasoning="Step 1: Extract Key Identifiers from Query Material. The query material is a 'Bearing Unit' manufactured by 'Link-Belt' with manufacturer part number 'FC-B22443E' (or FC-B 22443E) and a shaft diameter of 2-11/16 inches. Step 2: Systematic Comparison. I analyzed all 562 candidate materials provided. The candidates consist of a wide variety of industrial components, including different bearing types (Deep Groove Ball, Cylindrical Roller, Spherical Roller, Tapered Roller, Angular Contact, Thrust, and Needle bearings), V-belts, Oil Seals, and specific pump/machine parts (Impellers, Shafts, Gears, Valves). Step 3: Specific Part Number and Manufacturer Check. I specifically looked for the part number 'FC-B2

In [63]:
improved_llm_results

[{'case_number': 1,
  'query_pk': 'K011_K0060615920103',
  'has_answer': True,
  'answer_pk': ['7511_75060615969205'],
  'answer_in_candidates': True,
  'answer_rank': None,
  'llm_response': ImprovedMaterialGroupingResult(query_pk='K011_K0060615920103', status='success', matched_materials=[MatchedMaterial(pk='7511_75060615969205', confidence=0.95, reason="Both materials refer to the SKF cylindrical roller bearing model NU 1015. The query specifies 'ML' (machined brass cage), which aligns perfectly with the candidate's 'cage material brass' and 'NU 1015 m' designation. The standard ISO dimensions for NU 1015 (75x115x20mm) match the query's specific dimension notes.")], step_by_step_reasoning="Step 1: Extraction of Query Identifiers - The query material is a cylindrical roller bearing, model NU 1015 ML, manufactured by SKF, with dimensions 75x115x20mm (d/D/B) and a machined brass cage (indicated by the 'ML' suffix). Step 2: Candidate Screening - I reviewed all 562 candidates for the mod